# 모델 저장하기

- 무엇을 저장해야 할까?
    - 모델의 구조
    - 모델의 weights

## 훈련하는 동안 체크포인트 저장하기

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# 단순한 모델 정의
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

In [4]:
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### 체크포인트 콜백 사용하기

In [5]:
# 체크포인트 파일 경로 설정
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path) # 현재 경로 폴더 반환
checkpoint_dir

'training_1'

In [6]:
# 모델의 가중치를 저장하는 체크포인트 콜백 선언
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백 전달

Epoch 1/10
29/32 [==========================>...] - ETA: 0s - loss: 1.1659 - accuracy: 0.6713
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 2s 23ms/step - loss: 1.1228 - accuracy: 0.6840 - val_loss: 0.7125 - val_accuracy: 0.7940
Epoch 2/10
26/32 [=======================>......] - ETA: 0s - loss: 0.4302 - accuracy: 0.8774
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 13ms/step - loss: 0.4139 - accuracy: 0.8840 - val_loss: 0.5308 - val_accuracy: 0.8310
Epoch 3/10
28/32 [=========================>....] - ETA: 0s - loss: 0.2702 - accuracy: 0.9375
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.2744 - accuracy: 0.9350 - val_loss: 0.4801 - val_accuracy: 0.8560
Epoch 4/10
25/32 [======================>.......] - ETA: 0s - loss: 0.2129 - accuracy: 0.9525
Epoch 4: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - los

In [7]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트합니다  
 - 아키텍처가 동일한 모델 간에는 가중치를 공유 가능

 - 그렇다면, 원래 모델과 동일한 아키텍처를 가진 모델을 만든 다음 체크포인트 부여해보자.

In [8]:
# 동일한 아키텍처를 가진 모델 선언
model = create_model()

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3045 - accuracy: 0.1060 - 291ms/epoch - 9ms/step
훈련되지 않은 모델의 정확도: 10.60%


허접하다.

In [9]:
# 체크포인트에서 가중치를 로드하고 평가해보자.
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4196 - accuracy: 0.8740 - 91ms/epoch - 3ms/step
복원된 모델의 정확도: 87.40%


In [10]:
# format 리뷰
'온도 : {t:05d}도, 나이 : {a:03d}세'.format(t=10, a=13)

t=10
a=13
f'온도 : {t:05d}도, 나이 : {a:03d}세'

'온도 : 00010도, 나이 : 013세'

In [44]:
'{epoch:04d}, {batch:04d}'.format(batch=8, epoch=1)


batch=8
epoch=1
f'{epoch:04d}, {batch:04d}'

'0001, 0008'

In [12]:
# 파일 이름에 epoch 값 부여 (format 형식)
checkpoint_path = "training_2-1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

- 체크포인트 주요 파라미터

In [13]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 주기 설정 가능 ('epoch'=에폭마다, 숫자=해당 배치마다)
    )

In [14]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [15]:
model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 17ms/step - loss: 1.1712 - accuracy: 0.6620 - val_loss: 0.6909 - val_accuracy: 0.8010
Epoch 2/50
32/32 [==============================] - 0s 12ms/step - loss: 0.4364 - accuracy: 0.8730 - val_loss: 0.5724 - val_accuracy: 0.8210
Epoch 3/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2941 - accuracy: 0.9170 - val_loss: 0.4671 - val_accuracy: 0.8530
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2073 - accuracy: 0.9480 - val_loss: 0.4524 - val_accuracy: 0.8550
Epoch 5/50
32/32 [==============================] - 0s 12ms/step - loss: 0.1600 - accuracy: 0.9710 - val_loss: 0.4223 - val_accuracy: 0.8570
Epoch 6/50
32/32 [==============================] - 0s 11ms/step - loss: 0.1123 - accuracy: 0.9800 - val_loss: 0.4063 - val_accuracy: 0.8700
Epoch 7/50
32/32 [==============================] - 0s 9ms/step - loss: 0.0908 - accuracy: 0.9840 - val_loss: 0.4346 - val_accuracy: 0.8530
Epoch 8/50
32/

In [16]:
# 동일한 아키텍처를 가진 모델 선언
model = create_model()

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3586 - accuracy: 0.0960 - 178ms/epoch - 6ms/step
훈련되지 않은 모델의 정확도:  9.60%


In [17]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2-1/cp-0010.ckpt'

In [18]:
model.load_weights(latest)

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4030 - accuracy: 0.8760 - 54ms/epoch - 2ms/step
훈련되지 않은 모델의 정확도: 87.60%


In [19]:
### mytraining1 폴더에 저장
### best only로 저장, 최종 값만 저장되도록
### weight값만
checkpoint_path = "mytraining1/cp.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

model = create_model()

model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 10ms/step - loss: 1.1894 - accuracy: 0.6400 - val_loss: 0.7003 - val_accuracy: 0.7880
Epoch 2/50
32/32 [==============================] - 0s 7ms/step - loss: 0.4381 - accuracy: 0.8760 - val_loss: 0.5384 - val_accuracy: 0.8320
Epoch 3/50
32/32 [==============================] - 0s 7ms/step - loss: 0.2949 - accuracy: 0.9220 - val_loss: 0.4733 - val_accuracy: 0.8480
Epoch 4/50
32/32 [==============================] - 0s 6ms/step - loss: 0.2026 - accuracy: 0.9580 - val_loss: 0.4738 - val_accuracy: 0.8460
Epoch 5/50
32/32 [==============================] - 0s 7ms/step - loss: 0.1658 - accuracy: 0.9660 - val_loss: 0.4176 - val_accuracy: 0.8640
Epoch 6/50
32/32 [==============================] - 0s 7ms/step - loss: 0.1245 - accuracy: 0.9750 - val_loss: 0.4186 - val_accuracy: 0.8650
Epoch 7/50
32/32 [==============================] - 0s 6ms/step - loss: 0.0908 - accuracy: 0.9850 - val_loss: 0.4120 - val_accuracy: 0.8650
Epoch 8/50
32/32 [=

In [20]:
model.load_weights(checkpoint_path) # "mytraining1/cp.ckpt"

model.evaluate(test_images,  test_labels, verbose=2)

32/32 - 0s - loss: 0.4011 - accuracy: 0.8730 - 60ms/epoch - 2ms/step


[0.4011469781398773, 0.8730000257492065]

In [45]:
tf.keras.callbacks.ModelCheckpoint?

In [22]:
### mytraining2 폴더에 저장
### 5epoch마다 저장
### weight값만
checkpoint_path = "mytraining3/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=False, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    period=5, # 체크포인트 저장하는 에폭주기 설정 가능
    save_freq='epoch' # epoch의 끝에서 저장할지 여부
    )

In [23]:
model = create_model()

model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 10ms/step - loss: 1.1696 - accuracy: 0.6700 - val_loss: 0.6972 - val_accuracy: 0.7960
Epoch 2/50
32/32 [==============================] - 0s 6ms/step - loss: 0.4279 - accuracy: 0.8800 - val_loss: 0.5532 - val_accuracy: 0.8220
Epoch 3/50
32/32 [==============================] - 0s 6ms/step - loss: 0.2945 - accuracy: 0.9280 - val_loss: 0.4978 - val_accuracy: 0.8440
Epoch 4/50
32/32 [==============================] - 0s 7ms/step - loss: 0.2044 - accuracy: 0.9520 - val_loss: 0.4376 - val_accuracy: 0.8610
Epoch 5/50
32/32 [==============================] - 0s 6ms/step - loss: 0.1516 - accuracy: 0.9660 - val_loss: 0.4292 - val_accuracy: 0.8570
Epoch 6/50
32/32 [==============================] - 0s 7ms/step - loss: 0.1184 - accuracy: 0.9790 - val_loss: 0.4565 - val_accuracy: 0.8520
Epoch 7/50
32/32 [==============================] - 0s 6ms/step - loss: 0.0852 - accuracy: 0.9860 - val_loss: 0.4116 - val_accuracy: 0.8610
Epoch 8/50
32/32 [=

### early stopping & 체크포인트 같이 사용하기

In [24]:
checkpoint_path = "training_3/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [25]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

In [26]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [27]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [28]:
model.fit(train_images,
          train_labels,
          epochs=100,
          callbacks=[cp_callback, early], # 체크포인트 부여, early stopping
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/100
32/32 [==============================] - 1s 11ms/step - loss: 1.1548 - accuracy: 0.6690 - val_loss: 0.6944 - val_accuracy: 0.8090
Epoch 2/100
32/32 [==============================] - 0s 7ms/step - loss: 0.4264 - accuracy: 0.8800 - val_loss: 0.5109 - val_accuracy: 0.8440
Epoch 3/100
32/32 [==============================] - 0s 7ms/step - loss: 0.2856 - accuracy: 0.9220 - val_loss: 0.5143 - val_accuracy: 0.8380
Epoch 4/100
32/32 [==============================] - 0s 6ms/step - loss: 0.2094 - accuracy: 0.9590 - val_loss: 0.4553 - val_accuracy: 0.8540
Epoch 5/100
32/32 [==============================] - 0s 7ms/step - loss: 0.1528 - accuracy: 0.9700 - val_loss: 0.4860 - val_accuracy: 0.8440
Epoch 6/100
32/32 [==============================] - 0s 5ms/step - loss: 0.1169 - accuracy: 0.9760 - val_loss: 0.4556 - val_accuracy: 0.8430
Epoch 7/100
32/32 [==============================] - 0s 7ms/step - loss: 0.0869 - accuracy: 0.9820 - val_loss: 0.4052 - val_accuracy: 0.8600
Epoch 8/100


### 직접 가중치 체크포인트 저장

In [29]:
# 가중치 저장
model.save_weights('./checkpoints/my_checkpoint.ckpt')

model = create_model()

# 가중치 복원
model.load_weights('./checkpoints/my_checkpoint.ckpt')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4217 - accuracy: 0.8690 - 187ms/epoch - 6ms/step
복원된 모델의 정확도: 86.90%


## 전체 모델 저장(모델레이어와 함께 저장)
1) save_weights_only=False 로 변경  
2) 직접 model.save() : SavedModel(pb)  
3) HDF5 파일로 저장하기

### save_weights_only=False 로 변경

In [30]:
checkpoint_path = "training_4/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [31]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=False, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

In [32]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [33]:
model.fit(train_images,
          train_labels,
          epochs=1,
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

29/32 [==========================>...] - ETA: 0s - loss: 1.1948 - accuracy: 0.6541

32/32 [==============================] - 1s 35ms/step - loss: 1.1495 - accuracy: 0.6670 - val_loss: 0.7374 - val_accuracy: 0.7720


### SavedModel(pb)
SavedModel 형식은 protobuf 바이너리와 TensorFlow 체크포인트를 포함

In [34]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1494 - accuracy: 0.6770
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4252 - accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2838 - accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2207 - accuracy: 0.9390
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1539 - accuracy: 0.9670


In [35]:
# my_model 디렉토리
!ls saved_model

my_model


In [36]:
# assests 폴더, saved_model.pb, variables 폴더
!ls saved_model/my_model

assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


In [37]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 512)               401920    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있습니다. 이 모델을 평가하고 예측에 사용해 보죠:

In [38]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4168 - accuracy: 0.8580 - 203ms/epoch - 6ms/step
복원된 모델의 정확도: 85.80%
32/32 [==============================] - 0s 2ms/step
(1000, 10)


### HDF5 파일로 저장하기
케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다.

In [39]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 6ms/step - loss: 1.1543 - accuracy: 0.6760
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4253 - accuracy: 0.8810
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2789 - accuracy: 0.9300
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2184 - accuracy: 0.9420
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1560 - accuracy: 0.9670


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### h5 VS pb
- 공통점 : 레이어 & 가중치 모두 저장
- 차이점  
1) HDF5 : 객체 구성을 사용하여 모델 아키텍처를 저장  
2) SavedModel : 실행 그래프를 저장. 원본 코드 없이도 사용자 지정 객체를 저장 가능

#### h5 to pb 변환하기

In [40]:
!mkdir converted

In [41]:
model = tf.keras.models.load_model('my_model.h5')

In [42]:
export_path = 'converted'
model.save(export_path, save_format="tf")

### 과제
아래 조건에 맞춰 콜백을 이용하여 cifar 10 데이터를 resnet 50으로 학습을 시켜봅시다.

- early stopping : 50 에폭을 잡고, val loss 가 더이상 개선되지 않을 때 조기 종료 (세부 설정은 자율적으로 설정)
- model save : weights만 저장 (세부 설정을 자율)

학습이 완료되면, 저장된 weights를 불러와서 검증해봅시다.